In [ ]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import json
import geopandas as gpd
import pandas as pd
import numpy as np
from src import preparedata
from src import modelareaegpdcombined as modelarea
from src import inferenceegpd as inference
from src import savehazard
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

print(tf.__version__)
print(tf.config.list_physical_devices(device_type=None))
params = json.load(open("params/paramsegpdcombined.json", "r"))
weights = "savedweights/final_model20230823V1egpdcombined.h5"  # "checkpoints/"#"savedweights/final_model20230705V2.h5"

In [ ]:
su = gpd.read_file("Data/SlopeUnits/SlopeUnits_V3.shp")

In [ ]:
# check landslide size distribution
all_area = np.load("Data/Inventory/all_area.npy")

In [ ]:
np.quantile(all_area, [0.05, 0.5, 0.95])

In [ ]:
print((all_area < 0.40).mean())
print((all_area < 2.0).mean())
print((all_area < 10.0).mean())

In [ ]:
landslidehazard = modelarea.lhmodel(params["modelparam"])
landslidehazard.preparemodel()
hazcols = []


def extractRp(
    su=su,
    models=[0.4, 2.0, 10.0],  # [0.01760698, 0.42372807, 4.24535745],#,
    scenarios=["SSP245", "SSP585"],
    rps=[5, 10, 15, 20],
):
    first = True
    for model in models:
        for scenario in scenarios:
            for rp in rps:
                dataset = preparedata.readGPDData(params["dataprepinargs"])
                dataset.preparedataclimate(rp=rp, model="CIMP6", scenario=scenario)
                haz = inference.inferenceLHProb(
                    model=landslidehazard.model,
                    model_weights=weights,
                    xdata=dataset.Xinference.astype("float32"),
                    ep=model,
                )
                print(haz.shape)
                print(haz)
                ids = dataset.InferenceID
                hazcol = f"{rp}_{scenario}_{str(model).replace('.','_')}"
                if first:
                    first = False
                    hazarddata = pd.DataFrame({"cat": ids, hazcol: haz.flatten()})
                    # hazarddata=np.expand_dims(haz,axis=-1)
                else:
                    df = pd.DataFrame({hazcol: haz.flatten()})
                    hazarddata = pd.concat([hazarddata, df], axis=1, join="inner")
                    # hazarddata=np.concatenate((hazarddata,np.expand_dims(haz,axis=-1)),axis=-1)
                print(hazarddata.shape)

                #
                # df=pd.DataFrame({'cat':ids,hazcol:haz})
                # projections=su.merge(df,on='cat')
                hazcols.append(hazcol)
                # su=projections
                # break
            # break
        # break
    return hazarddata

In [ ]:
haz = extractRp()

In [ ]:
haz.columns = ["cat"] + hazcols
haz = haz.drop_duplicates(subset=["cat"])

In [ ]:
results = su.merge(haz, on="cat")
results.to_file("Data/ResultsV5/ClimateProjectionsV555555ALT_prob.gpkg", driver="GPKG")

In [ ]:
results

# predict current situations

In [ ]:
landslidehazard = modelarea.lhmodel(params["modelparam"])
landslidehazard.preparemodel()
hazcols = []
rps = [5, 10, 15, 20]
first = True
models = [0.4, 2.0, 10.0]  # [0.01760698, 0.42372807, 4.24535745]#[5,50,95]
for model in models:
    for rp in rps:
        dataset = preparedata.readGPDData(params["dataprepinargs"])
        dataset.preparedatainference(rp)
        haz = inference.inferenceLHProb(
            model=landslidehazard.model,
            model_weights=weights,
            xdata=dataset.Xinference.astype("float32"),
            ep=model,
        )
        ids = dataset.InferenceID
        hazcol = f"{rp}_{str(model).replace('.','_')}"
        if first:
            first = False
            hazarddatacr = pd.DataFrame({"cat": ids, hazcol: haz.flatten()})
            # hazarddata=np.expand_dims(haz,axis=-1)
        else:
            df = pd.DataFrame({hazcol: haz.flatten()})
            hazarddatacr = pd.concat([hazarddatacr, df], axis=1, join="inner")

In [ ]:
current_haz = su.merge(hazarddatacr, on="cat")
current_haz = current_haz.drop_duplicates(subset=["cat"])
current_haz.to_file(
    "Data/ResultsV5/Current_predictionsV5555ALT_prob.gpkg", driver="GPKG"
)

# Combine all results to save

In [ ]:
results.columns

In [ ]:
["cat"] + current_haz.columns[3:].to_list()

In [ ]:
result_sub = results
all_results = result_sub.merge(
    current_haz[["cat"] + current_haz.columns[3:].to_list()], on="cat"
)

for quantile in [0.4, 2.0, 10.0]:  # [0.01760698, 0.42372807, 4.24535745]:
    for year in ["5", "10", "15", "20"]:
        all_results[f"r_SSP245_{year}_{str(quantile).replace('.','_')}"] = 100 * (
            (
                all_results[
                    f"{year}_SSP245_{str(quantile).replace('.','_')}"
                ].to_numpy()
                - all_results[f"{year}_{str(quantile).replace('.','_')}"].to_numpy()
            )
            / (all_results[f"{year}_{str(quantile).replace('.','_')}"].to_numpy())
        )
        all_results[f"r_SSP585_{year}_{str(quantile).replace('.','_')}"] = 100 * (
            (
                all_results[
                    f"{year}_SSP585_{str(quantile).replace('.','_')}"
                ].to_numpy()
                - all_results[f"{year}_{str(quantile).replace('.','_')}"].to_numpy()
            )
            / (all_results[f"{year}_{str(quantile).replace('.','_')}"].to_numpy())
        )

In [ ]:
all_results.replace([np.inf, -np.inf], 0, inplace=True)
all_results.replace([np.nan, -np.nan], 0, inplace=True)

In [ ]:
all_results.to_file("Data/ResultsV5/all_resultsV56666666666ALT_prob.gpkg")